# Import Packages 

In [56]:
import tweepy
import csv
import pandas as pd
import pickle
import json
import jsonlines

# Dumping responses into JSONlines file

In [57]:
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'                                  #user's details for dev.twitter.
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'      
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)                   #checkng for authentication of users.
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

all_data={}                                                                 #dictionary that will contains all the 
                                                                            #tweet fetch from id.
count=0
for i in tweepy.Cursor(api.user_timeline, id="midasiiitd",tweet_mode='extended').items(): #"cursor handle all the pagination
    all_data[count]=i._json                                                               #(till last page it fetch all the tweet)."
    count+=1
with jsonlines.open('all_tweet', mode='w') as writer:
    writer.write(all_data)

# Parsing JSONline

In [61]:
#In Parsing I select particular things from all_file 
#which is necessary for examp. ["Tweet's Text", "Date and Time", "favorites/likes count", "Retweet count", "Image count"]
with jsonlines.open('all_tweet') as reader:
    for all_data in reader:   
        tweet_keys = all_data.keys()
        no_of_image=[]
        for i in tweet_keys:
            img=[]
            k=all_data[i].keys()
            if "retweeted_status" in k:
                k1 = all_data[i]["retweeted_status"].keys()
                if "extended_entities" in k1:
                    length1 = len(all_data[i]["retweeted_status"]["extended_entities"]["media"])
                    val=0
                    for l in range(length1):
                        if all_data[i]["retweeted_status"]["extended_entities"]["media"][l]["type"]=="photo":
                            val+=1          
                else:
                    val=0
                img.append(all_data[i]["full_text"])
                img.append(all_data[i]["created_at"])
                img.append(all_data[i]["favorite_count"])
                img.append(all_data[i]["retweet_count"])
                img.append(val)

            elif "extended_entities" in k:
                length2 = len(all_data[i]["extended_entities"]["media"])
                val=0
                for l in range(length2):                    
                    if all_data[i]["extended_entities"]["media"][l]["type"]=="photo":
                        val+=1
                img.append(all_data[i]["full_text"])
                img.append(all_data[i]["created_at"])
                img.append(all_data[i]["favorite_count"])
                img.append(all_data[i]["retweet_count"])
                img.append(val)

            else:
                val=0
                img.append(all_data[i]["full_text"])
                img.append(all_data[i]["created_at"])
                img.append(all_data[i]["favorite_count"])
                img.append(all_data[i]["retweet_count"])
                img.append(val)
            no_of_image.append(img)

# Method for adjusting text allignment 

In [66]:
#tweet_text_format decides after how many character nextline charachter should be introduced to make sure the 
#correct adjustmet of the tweet_text in the table.
def tweet_text_fomat(text, line_len):
    tokens = text.split(" ")
    new_text = ""
    temp_len=0
    for i in tokens:
        if (temp_len+(len(i))<=line_len-1):
            new_text=new_text+i+" "
            temp_len=temp_len+len(i)+1   #+1 to deal with '/0'
        else:
            new_text=new_text+"\n"+i+" "
            temp_len=len(i)+1
    return new_text

# Display the tweets details in a tabular format.

In [68]:
import prettytable
from prettytable import ALL as ALL
x = prettytable.PrettyTable(hrules=ALL)
x.field_names = ["Tweet's Text", "Date and Time", "favorites/likes count", "Retweet count", "Image count"]
for i in no_of_image:
    i[0] = tweet_text_fomat(i[0], 30)                                
    x.add_row([i[0], i[1], i[2],i[3],i[4]])

# Output

In [69]:
print(x)                                               

+------------------------------------+--------------------------------+-----------------------+---------------+-------------+
|            Tweet's Text            |         Date and Time          | favorites/likes count | Retweet count | Image count |
+------------------------------------+--------------------------------+-----------------------+---------------+-------------+
|      RT @IEEEBigMM19: We are       | Sun Mar 24 06:23:37 +0000 2019 |           0           |       4       |      0      |
|                                    |                                |                       |               |             |
|     honored to have Dr. Chang      |                                |                       |               |             |
|                Wen                 |                                |                       |               |             |
|     Chen (Chinese University       |                                |                       |               |       